In [1]:
## Train for 50 epochs. First use only fine tuning on last layer
## Then compare with fine-tuning entire network
## TODO: 
## 0. Pre-process images
##      Resize images
##      Other pre-processing
## 1. Get mean and std of dataset
## 2. Train on Resnet-34
## 3. Train on efficient net b3
## 4. Train on mobilenet v3 large
## 5. Write a script to plot loss + accuracy graph
## 6. Get FLOPs - done
## 7. Get num layers - done
## 8. Add class weights
## 9. Implement gradcam

In [42]:
import sys
sys.path.append('../src/')

In [43]:
%load_ext autoreload
%autoreload 2

from multilabel.train import train_model
from model import initialize_model
from utils import set_requires_grad, save_model
from multilabel.data import load_data
from plotting import plot_data_loader
from multilabel.eval import eval_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
import os
import copy
import time
import random
import pickle

import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from tqdm import tqdm

from sklearn.metrics import f1_score, confusion_matrix
from numpy.random import shuffle
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision.io import read_image
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import torch.autograd.profiler as tprofiler
import torch.utils.data as td

plt.rcParams["savefig.bbox"] = 'tight'

In [45]:
seed = 42

random.seed(seed)

# pytorch RNGs
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
np.random.seed(seed)

In [46]:
data_dir = "../data"
images_dir = "../data/processed"

In [47]:
# Get best num_workers
# for i in range(97):
#     start = time.time()
#     data_loader = load_data(images_dir,
#                                                                    batch_size = 96, 
#                                                                    input_size = 299, 
#                                                                    norm_arr = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
#                                                                    num_workers = i)
#     iter(data_loader['train']).next()[0].shape
#     print(f"{i}: {time.time()-start}")

In [48]:
# Check if GPU is available.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Models options: resnet50, resnet34, inceptionv3, vgg16, mobile_net_v3_large, efficient_net_b1, efficient_net_b0, efficient_net_b3.
model_name = "efficient_net_b1"

# Number of classes.
num_classes = 20

# Batch Size.
batch_size = 32

# Epochs to train for.
num_epochs = 50

# Number of workers for data loader.
num_workers = 12

# Imagenet norm array passed as default value.
# norm_arr=([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# Chest x-ray8 training dataset metrics 
norm_arr=([0.4955, 0.4955, 0.4955], [0.2892, 0.2892, 0.2892])


# Feature extract flag: False - Tune the whole model,
#                       True - Update only the reshaped layer parameters.
feature_extract = False

# Use pretrained flag: None - Use random weights
#                      String - Use pretrained weights given by String
use_pretrained = None

# Initialize the model for this run.
model_pyt, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=use_pretrained)

# lr start and end points for training.
lr_start = 0.01
lr_end = 0.001

# Print the model we just instantiated
#print(model_ft)

# Positive class weights.
#pos_weight=torch.as_tensor([2., 3., 6., 6., 3., 7., 9.], dtype=torch.float)
#pos_weight = pos_weight.to(device)

cuda:0


In [49]:
data_loaders = load_data(images_dir,
                         batch_size = batch_size, 
                         input_size = input_size, 
                         norm_arr = norm_arr,
                         num_workers = num_workers)

In [50]:
data_loaders['train'].dataset.classes

['Active',
 'Alert',
 'Amazed',
 'Amused',
 'Calm',
 'Cheerful',
 'Confident',
 'Conscious',
 'Creative',
 'Eager',
 'Educated',
 'Emotional',
 'Fashionable',
 'Feminine',
 'Inspired',
 'Loving',
 'Manly',
 'Persuaded',
 'Thrifty',
 'Youthful']

In [51]:
# plot_data_loader(data_loaders['train'], (2,2))

plot_data_loader(data_loader['train'], (2,2))

plot_data_loader(data_loader['test'], (2,2))

plot_data_loader(data_loader['val'], (2,2))

In [52]:
# Send model to GPU
model_pyt = model_pyt.to(device)

# Find parameters to be updated in this run.
# parameters with requires_grad = True.
params_to_update = model_pyt.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_pyt.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_pyt.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

Params to learn:
	 features.0.0.weight
	 features.0.1.weight
	 features.0.1.bias
	 features.1.0.block.0.0.weight
	 features.1.0.block.0.1.weight
	 features.1.0.block.0.1.bias
	 features.1.0.block.1.fc1.weight
	 features.1.0.block.1.fc1.bias
	 features.1.0.block.1.fc2.weight
	 features.1.0.block.1.fc2.bias
	 features.1.0.block.2.0.weight
	 features.1.0.block.2.1.weight
	 features.1.0.block.2.1.bias
	 features.1.1.block.0.0.weight
	 features.1.1.block.0.1.weight
	 features.1.1.block.0.1.bias
	 features.1.1.block.1.fc1.weight
	 features.1.1.block.1.fc1.bias
	 features.1.1.block.1.fc2.weight
	 features.1.1.block.1.fc2.bias
	 features.1.1.block.2.0.weight
	 features.1.1.block.2.1.weight
	 features.1.1.block.2.1.bias
	 features.2.0.block.0.0.weight
	 features.2.0.block.0.1.weight
	 features.2.0.block.0.1.bias
	 features.2.0.block.1.0.weight
	 features.2.0.block.1.1.weight
	 features.2.0.block.1.1.bias
	 features.2.0.block.2.fc1.weight
	 features.2.0.block.2.fc1.bias
	 features.2.0.block.2.fc

In [53]:
# 17 min 1 epoch - 128 batch size - inception
# Efficientnet b0 - batch 96 - epoch 50 - num_workers 2 - flip, auto cont, sharp - 

In [54]:
# Observe that all parameters are being optimized
optimizer = optim.Adam(params_to_update, lr=lr_start)

# Learning rate scheduler.
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=lr_end,
                                                 last_epoch=-1)


# Setup the loss fxn
# pos_weight=pos_weight
criterion = nn.BCEWithLogitsLoss()

# Train and evaluate
model_pyt, prof, val_history, train_history = train_model(device, model_pyt, data_loaders, 
                                                            optimizer, scheduler,
                                                            criterion, 
                                                            num_epochs=num_epochs,
                                                            num_classes=num_classes,
                                                            is_inception=(model_name=="inceptionv3"),
                                                            profiler=False)

Epoch 1/100
----------


RuntimeError: Caught RuntimeError in DataLoader worker process 5.
Original Traceback (most recent call last):
  File "c:\Users\ro_chopr\.conda\envs\dl\lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "c:\Users\ro_chopr\.conda\envs\dl\lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\ro_chopr\.conda\envs\dl\lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\ro_chopr\Documents\processing-ads\notebooks\../src\multilabel\loader.py", line 22, in __getitem__
    image = self.transform(image)
  File "c:\Users\ro_chopr\.conda\envs\dl\lib\site-packages\torchvision\transforms\transforms.py", line 95, in __call__
    img = t(img)
  File "c:\Users\ro_chopr\.conda\envs\dl\lib\site-packages\torch\nn\modules\module.py", line 

In [ ]:
dataset_name = 'chest_xray_8'

In [ ]:
save_model(model_pyt.state_dict(), '../../models/', 
           f'{model_name}_{dataset_name}_{num_epochs}_{batch_size}_{lr_start}_{lr_end}_model_weights.pth')

True

In [15]:
eval_metrics = eval_model(device=device, model=model_pyt, test_loader=data_loaders['test'], is_inception=(model_name=="inceptionv3"), num_classes=7)

In [16]:
for i, v in eval_metrics.items():
    print(f"{i}: {v}")

acc: 0.77413485079071
f1: 0.4459599730627861
cm: [[[2708  565]
  [ 326  719]]

 [[2864  409]
  [ 403  642]]

 [[2946  813]
  [ 228  331]]

 [[2982  681]
  [ 387  268]]

 [[2695  576]
  [ 552  495]]

 [[3250  623]
  [ 228  217]]

 [[2986  866]
  [ 170  296]]]
outputs: [[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
targets: [[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [17]:
with open(f'../../models/val_history_{model_name}_{dataset_name}_{num_epochs}_{batch_size}_{lr_start}_{lr_end}_.pickle', 'wb') as handle:
    pickle.dump(val_history, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'../../models/train_history_{model_name}_{dataset_name}_{num_epochs}_{batch_size}_{lr_start}_{lr_end}_.pickle', 'wb') as handle:
    pickle.dump(train_history, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'../../models/eval_metrics_{model_name}_{dataset_name}_{num_epochs}_{batch_size}_{lr_start}_{lr_end}_.pickle', 'wb') as handle:
    pickle.dump(eval_metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('filename.pickle', 'rb') as handle:
#    b = pickle.load(handle)

In [ ]:
#print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=50))

In [9]:
# Plot the training curves of validation accuracy vs. number
#  of training epochs for the transfer learning method and
#  the model trained from scratch
# vhist = []
# vhist = [h.cpu().numpy() for h in val_acc_history]
# thist = []
# thist = [h.cpu().numpy() for h in train_acc_history]

# plt.title("Accuracy vs. Number of Training Epochs")
# plt.xlabel("Training Epochs")
# plt.ylabel("Accuracy")
# #plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
# plt.plot(range(1,num_epochs+1),vhist,label="Validation")
# plt.plot(range(1,num_epochs+1),thist,label="Training")
# plt.ylim((0,1.))
# plt.xticks(np.arange(1, num_epochs+1, 1.0))
# plt.legend()
# plt.show()

In [8]:
# Plot the training curves of validation accuracy vs. number
#  of training epochs for the transfer learning method and
#  the model trained from scratch
# vhist = []
# vhist = [h for h in val_loss_history]
# thist = []
# thist = [h for h in train_loss_history]

# plt.title("Loss vs. Number of Training Epochs")
# plt.xlabel("Training Epochs")
# plt.ylabel("Loss")
# #plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
# plt.plot(range(1,num_epochs+1),vhist,label="Validation")
# plt.plot(range(1,num_epochs+1),thist,label="Training")
# plt.ylim((0,1.))
# plt.xticks(np.arange(1, num_epochs+1, 1.0))
# plt.legend()
# plt.show()